In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.2MB/s 
     |████████████████████████████████| 901kB 56.2MB/s 
     |████████████████████████████████| 3.3MB 55.2MB/s 


In [ ]:
!unzip model_save.zip

Archive:  model_save.zip
   creating: model_save/
  inflating: model_save/config.json  
  inflating: model_save/pytorch_model.bin  
  inflating: model_save/special_tokens_map.json  
  inflating: model_save/tokenizer_config.json  
  inflating: model_save/vocab.txt    


In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification


class BERT:
    def set_device(self):
        """
        Sets device as 'GPU' or 'CPU'
        """
        # if there's a GPU available...
        if torch.cuda.is_available():
            # tell PyTorch to use the GPU.
            device = torch.device('cuda')
        else:
            device = torch.device('cpu')

        return device

    def load_model(self, path):
        """
        Loads model and tokenizer from binary file

        path: str
        """
        # load a trained model and vocabulary that you have fine-tuned
        model = BertForSequenceClassification.from_pretrained(path)
        tokenizer = BertTokenizer.from_pretrained(path)

        return model, tokenizer

    def predict(self, text, model_path):
        """
        Uses fine tuned BERTforSequenceClassification to make a 
        prediction
        
        text: str
        model_path: str
        """
        device = self.set_device()

        model, tokenizer = self.load_model(model_path)

        model.to(device)

        # prepare our text into tokenized sequence
        inputs = tokenizer(text, padding=True, truncation=True,
                           max_length=64, return_tensors='pt').to(device)

        # perform inference on our model
        outputs = model(**inputs)
        
        # get output probabilities through softmax
        probs = outputs[0].softmax(1)
        
        # executing argmax function to get label
        return np.argmax(probs.detach().cpu().numpy())


In [ ]:
bert = BERT()

test = "the police shot and killed a 15 year old Black girl in Columbus as the verdict was being read. She reportedly called them for help. this isn't a giant step anywhere"

print(bert.predict(test, '/content/model_save'))

1
